# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Create reference to CSV file
csv_path = "weather_data.csv"
# Import the CSV into a pandas DataFrame
cities_weather_df = pd.read_csv(csv_path, low_memory=False)
cities_weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Torbay,47.6666,-52.7314,26.01,80,75,23.02,CA,1612996164
1,1,Rikitea,-23.1203,-134.9692,78.62,76,18,14.45,PF,1612996164
2,2,Lagoa,39.0500,-27.9833,57.33,69,68,35.43,PT,1612995896
3,3,Albany,42.6001,-73.9662,25.00,49,99,4.00,US,1612996073
4,4,Jamestown,42.0970,-79.2353,21.99,58,90,3.44,US,1612996135
...,...,...,...,...,...,...,...,...,...,...
548,548,Umarkot,25.3616,69.7362,56.86,33,30,1.48,PK,1612996402
549,549,Vytegra,61.0064,36.4481,-4.05,90,100,3.49,RU,1612996402
550,550,São Filipe,14.8961,-24.4956,72.01,76,50,11.14,CV,1612996402
551,551,Aginskoye,51.1000,114.5300,12.13,95,100,1.97,RU,1612996402


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Google developer API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = cities_weather_df[["Lat", "Lng"]]
# Fill NaN values and convert to float
humidity = cities_weather_df["Humidity"]
# Plot Heatmap
fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Drop cities where Max Temp less than 80 degrees Fahrenheit
cities_weather_over_80 = cities_weather_df.drop(cities_weather_df[cities_weather_df['Max Temp'] < 80].index, inplace = False)
# Drop cities where Max Temp greater than 85 degrees Fahrenheit
cities_weather_under_85 = cities_weather_over_80.drop(cities_weather_over_80[cities_weather_over_80['Max Temp'] > 85].index, inplace = False)
# Drop cities where Wind Speed is higher than 10 mph
cities_weather_low_winds = cities_weather_under_85.drop(cities_weather_under_85[cities_weather_under_85['Wind Speed'] < 10].index, inplace = False)
# Isolate cities with cloudy skies (cloudiness not equal to zero)
not_ideal_cities = cities_weather_low_winds[cities_weather_low_winds['Cloudiness'] != 0 ].index
# Delete these row indexes from dataFrame
cities_weather_low_winds.drop(not_ideal_cities , inplace=True)
cities_weather_low_winds


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
108,108,Broken Hill,-31.9500,141.4333,82.40,34,0,20.71,AU,1612996172
122,122,Faanui,-16.4833,-151.7500,81.70,71,0,10.98,PF,1612996175
156,156,São João da Barra,-21.6403,-41.0511,80.60,61,0,13.80,BR,1612996177
434,434,Lázaro Cárdenas,17.9583,-102.2000,80.02,63,0,10.20,MX,1612995853
453,453,Acapulco de Juárez,16.8634,-99.8901,82.96,60,0,11.56,MX,1612996077
507,507,Barra de Tecoanapa,16.5167,-98.7500,82.40,66,0,11.12,MX,1612996399


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = cities_weather_low_winds
hotel_df["Hotel Name"] = ""
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
108,108,Broken Hill,-31.9500,141.4333,82.40,34,0,20.71,AU,1612996172,
122,122,Faanui,-16.4833,-151.7500,81.70,71,0,10.98,PF,1612996175,
156,156,São João da Barra,-21.6403,-41.0511,80.60,61,0,13.80,BR,1612996177,
434,434,Lázaro Cárdenas,17.9583,-102.2000,80.02,63,0,10.20,MX,1612995853,
453,453,Acapulco de Juárez,16.8634,-99.8901,82.96,60,0,11.56,MX,1612996077,
507,507,Barra de Tecoanapa,16.5167,-98.7500,82.40,66,0,11.12,MX,1612996399,


In [36]:
# target_coordinates = "20.5022, -10.0711"
# target_radius = 45000
# target_type = "hotel"
# Jones bar
# Chinguetti

# 5.1033, 73.0708
# Eydhafushi
# Eydhafushi

# 20.5169, -13.0499
# Atar
# Atar

# geocoordinates
target_coordinates = "20.5169, -13.0499"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

# convert response to json
places_data = response.json()

# Print the json (pretty printed)
# print(json.dumps(places_data, indent=4, sort_keys=True))
# Print the name and address of the first restaurant that appears
print(places_data["results"][0]["name"])
print(places_data["results"][0]["vicinity"])

Atar
Atar


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel Info: {hotel_info}"])
fig.add_layer(markers)
fig

# Display figure


IndexError: list index out of range